In [1]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE\3. Trimestre\Venture Lab & Capstone\Capstone\Tech side\vl_optimizer


## Create a merged dataframe with the required information for the optimization

Import the forecasted data and add it to merged_data

In [2]:
import pandas as pd
import os
import data.data_utils as du

def join_csv_on_time_column(directory='.'):
    # Get all CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
    # Initialize an empty dataframe
    merged_df = pd.DataFrame()
    
    # Iterate through each CSV file and merge them on the 'Time' column
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))
        
        df = df[['Time', 'Forecasting Values']]
        
        column_name = csv_file[:-21]
        
        df[column_name] = df['Forecasting Values']
        
        df = df.drop('Forecasting Values', axis=1)  
        
        # Check if 'Time' column exists in the dataframe
        if 'Time' not in df.columns:
            print(f"'Time' column not found in {csv_file}")
            continue
        
        # Merge the dataframes
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Time', how='outer')
    
    return merged_df

# Run the function and store the result
merged_dataframe = join_csv_on_time_column(directory='data/forecasted_data')

Add the data from the processed dataset and adapt the naming of certain columns

In [3]:
processed_df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-09T19-25-47_industrial_site2_processed.csv')
merged_dataframe = pd.merge(merged_dataframe, processed_df[['Time', 'heat_index', 'price_mWh']], on='Time', how='inner')
merged_dataframe['Hour'] = merged_dataframe['Time'].apply(lambda x: int(x.split(' ')[1].split(':')[0]))

merged_dataframe['efficiency_adjusted_power'] = merged_dataframe['chiller_efficiency']
merged_dataframe = merged_dataframe.drop('chiller_efficiency', axis=1)

merged_dataframe['technological_centers_consumption'] = merged_dataframe['08 Technological Centers_Electric_Active Energy (kWh)']
merged_dataframe = merged_dataframe.drop('08 Technological Centers_Electric_Active Energy (kWh)', axis=1)

Check the nulls and solve by a linear interpolation

In [4]:
merged_dataframe.isnull().sum().sum(axis = 0)

6

In [5]:
merged_dataframe  = merged_dataframe.interpolate(method='linear', axis=0)
merged_dataframe.isnull().sum().sum(axis = 0)

0

Save the model

In [6]:
# Save the result to a new CSV file
merged_dataframe.to_csv('optimizer/optimizer_input/parameters.csv', index=False)